Notebook for testing random things

In [1]:
from cmaqpy.runcmaq import CMAQModel
import datetime
import pandas as pd
import xarray as xr

In [2]:
# Specify the start/end times
start_datetime = 'August 05, 2016'  # first day that you want for MCIP/CCTM
end_datetime = 'August 15, 2016'  # add one day to the final day that you want to run MCIP/CCTM
# Specify if you want to run the 12 km or the 4 km domain
appl = '2016_12OTC2'
# appl = '2016_4OTC2'
# Define the coordinate name (must match that in GRIDDESC)
coord_name = 'LAM_40N97W'
if appl == '2016_12OTC2':
    grid_name = '12OTC2'
elif appl == '2016_4OTC2':
    grid_name = '4OTC2'

# Create a CMAQModel object
cmaq_sim = CMAQModel(start_datetime, end_datetime, appl, coord_name, grid_name, verbose=True)

Application name: 2016_12OTC2
Coordinate name: LAM_40N97W
Grid name: 12OTC2
CCTM RUNID: v533_gcc9.3.1_2016_12OTC2
CMAQ run starting on: 2016-08-05 00:00:00
CMAQ run ending on: 2016-08-15 00:00:00


In [3]:
cmaq_sim.delt.days

10

In [5]:
start_datetimes_lst = [single_date for single_date in (cmaq_sim.start_datetime + datetime.timedelta(n) for n in range(cmaq_sim.delt.days))]
end_datetimes_lst = [single_date for single_date in (cmaq_sim.start_datetime + datetime.timedelta(n + 1) for n in range(cmaq_sim.delt.days))]

In [8]:
start_datetimes_lst[0]

datetime.datetime(2016, 8, 5, 0, 0)

In [45]:
smk_dates = pd.read_csv('/share/mzhang/jas983/cmaq_data/CMAQ_v5.3.3/2016_12OTC2/input/emis/smk_merge_dates_201608.txt', index_col=0, parse_dates=[0], infer_datetime_format=True)

In [51]:
s = smk_dates['  mwdss_N']
rep_day = s[start_datetimes_lst[0]]

20160809

In [74]:
def get_rep_dates(smk_dates_dir, dates_list, date_type='  mwdss_N'):
    """
    Get representative dates from the files produced by smkmerge.

    Make sure that you include the correct white space in the date_type
    parameter. The options are: 
    ' aveday_N', ' aveday_Y', 
    '  mwdss_N', '  mwdss_Y', 
    '   week_N', '   week_Y', 
    '      all'

    NOTE: this woudld be slightly faster if I didn't reopen the file each time, 
    but it's such a small fraction of the time for CCTM that I'm not
    going to worry about it now.
    """
    rep_days = []
    # Loop through each day in the input list and append the respective representative day to the list
    for date in dates_list:
        d_str = date.strftime("%Y%m")
        smk_dates = pd.read_csv(f'{smk_dates_dir}/smk_merge_dates_{d_str}.txt', index_col=0, parse_dates=[0], infer_datetime_format=True)
        s = smk_dates[date_type]
        rep_days.append(s[date])

    # Remove duplicates in the represenatitive days 
    result = [] 
    [result.append(x) for x in rep_days if x not in result]

    # Convert to datetimes
    result = pd.to_datetime(result, format='%Y%m%d')   
     
    return result

In [59]:
get_rep_dates('/share/mzhang/jas983/cmaq_data/CMAQ_v5.3.3/2016_12OTC2/input/emis', start_datetimes_lst, date_type='  mwdss_N')

[20160809,
 20160813,
 20160814,
 20160808,
 20160809,
 20160809,
 20160809,
 20160809,
 20160813,
 20160814]

In [75]:
get_rep_dates('/share/mzhang/jas983/cmaq_data/CMAQ_v5.3.3/2016_12OTC2/input/emis', start_datetimes_lst, date_type='  mwdss_Y')

DatetimeIndex(['2016-08-09', '2016-08-13', '2016-08-14', '2016-08-08'], dtype='datetime64[ns]', freq=None)

In [71]:
dates = pd.to_datetime(result, format='%Y%m%d')

In [72]:
dates

DatetimeIndex(['2016-08-09', '2016-08-13', '2016-08-14', '2016-08-08'], dtype='datetime64[ns]', freq=None)

In [73]:
for date in dates:
    print(date.strftime("%y%m%d"))

160809
160813
160814
160808


In [8]:
cmd = ''
for date in start_datetimes_lst:
    cmd = cmd + f'; *{date.strftime("%y%m%d")}'

In [9]:
cmd

'; *160805; *160806; *160807; *160808; *160809; *160810; *160811; *160812; *160813; *160814'

In [ ]:
mcip_sdatestr = mcip_start_datetime.strftime("%y%m%d")

In [17]:
for day_no in range(cmaq_sim.delt.days):
    start_datetime = cmaq_sim.start_datetime + datetime.timedelta(day_no)
    end_datetime = cmaq_sim.start_datetime + datetime.timedelta(day_no + 1)
    print(day_no)
    print(start_datetime)
    print(end_datetime)

0
2016-08-05 00:00:00
2016-08-06 00:00:00
1
2016-08-06 00:00:00
2016-08-07 00:00:00
2
2016-08-07 00:00:00
2016-08-08 00:00:00
3
2016-08-08 00:00:00
2016-08-09 00:00:00
4
2016-08-09 00:00:00
2016-08-10 00:00:00
5
2016-08-10 00:00:00
2016-08-11 00:00:00
6
2016-08-11 00:00:00
2016-08-12 00:00:00
7
2016-08-12 00:00:00
2016-08-13 00:00:00
8
2016-08-13 00:00:00
2016-08-14 00:00:00
9
2016-08-14 00:00:00
2016-08-15 00:00:00
